In [1]:
using HTTP, JSON, DataFrames

In [113]:
# base url for all FPL API endpoints
base_url="https://fantasy.premierleague.com/api/"

resp = HTTP.get(string(base_url,"bootstrap-static/"))
str = String(resp.body)
h = JSON.Parser.parse(str)

Dict{String, Any} with 8 entries:
  "events"        => Any[Dict{String, Any}("most_transferred_in"=>1, "chip_play…
  "total_players" => 9167407
  "phases"        => Any[Dict{String, Any}("name"=>"Overall", "stop_event"=>38,…
  "element_types" => Any[Dict{String, Any}("squad_select"=>2, "plural_name"=>"G…
  "teams"         => Any[Dict{String, Any}("points"=>0, "win"=>0, "strength_att…
  "elements"      => Any[Dict{String, Any}("points_per_game"=>"2.5", "squad_num…
  "game_settings" => Dict{String, Any}("squad_total_spend"=>1000, "squad_squads…
  "element_stats" => Any[Dict{String, Any}("label"=>"Minutes played", "name"=>"…

In [101]:
function rename_all(df,pre)
    rn_dict=Dict{Symbol,Symbol}()
    for n in names(df)
        push!(rn_dict,Symbol(n)=>Symbol(string(pre,"_",n)))
    end
    rename!(df,rn_dict)
    return df
end

rename_all (generic function with 1 method)

In [138]:
players=h["elements"]
players_df=vcat(DataFrame.(players)...);
rename_all(players_df,"Pl")
teams=h["teams"]
teams_df=vcat(DataFrame.(teams)...);
rename_all(teams_df,"Te")
#rename!(teams_df,Dict(:name=>:team_name,:code=>:team_code_,:form=>:team_form))
positions=h["element_types"]
for p in positions; delete!(p,"sub_positions_locked"); end
positions_df=vcat(DataFrame.(positions)...);
rename_all(positions_df,"Po");

In [139]:
merged_df=innerjoin(players_df,teams_df,on = :Pl_team=>:Te_id);
merged_df=innerjoin(merged_df,positions_df,on = :Pl_element_type=>:Po_id)

merged_df[:, ["Pl_first_name","Pl_second_name","Te_name","Po_singular_name_short"]]

,Pl_first_name,Pl_second_name,Te_name,Po_singular_name_short
,String,String,String,String
1,Bernd,Leno,Arsenal,GKP
2,Rúnar Alex,Rúnarsson,Arsenal,GKP
3,Willian,Borges Da Silva,Arsenal,MID
4,Pierre-Emerick,Aubameyang,Arsenal,FWD
5,Cédric,Soares,Arsenal,DEF
6,Alexandre,Lacazette,Arsenal,FWD
7,Granit,Xhaka,Arsenal,MID
8,Pablo,Marí,Arsenal,DEF
9,Héctor,Bellerín,Arsenal,DEF


In [163]:
merged_df[==("Salah").(merged_df.Pl_second_name),:][:, ["Pl_id","Pl_first_name","Pl_second_name","Te_name","Po_singular_name_short"]]

,Pl_id,Pl_first_name,Pl_second_name,Te_name,Po_singular_name_short
,Int64,String,String,String,String
1,233,Mohamed,Salah,Liverpool,MID


In [150]:
function get_gameweek_history(player_id)
    base_url="https://fantasy.premierleague.com/api/"

    resp = HTTP.get(string(base_url,"element-summary/",player_id,"/"))
    str = String(resp.body)
    _h = JSON.Parser.parse(str)
    out_df=vcat(DataFrame.(_h["history"])...);
    return out_df
end

function get_season_history(player_id)
    base_url="https://fantasy.premierleague.com/api/"

    resp = HTTP.get(string(base_url,"element-summary/",player_id,"/"))
    str = String(resp.body)
    _h = JSON.Parser.parse(str)
    out_df=vcat(DataFrame.(_h["history_past"])...);
    return out_df
end


get_season_history (generic function with 1 method)

In [164]:
get_gameweek_history(233)[:,["round","total_points","minutes","goals_scored","assists","value"]]
#get_season_history(233)[:,["season_name","total_points","minutes","goals_scored","assists"]]

,round,total_points,minutes,goals_scored,assists,value
,Int64,Int64,Int64,Int64,Int64,Int64
1,1,17,90,1,2,125
2,2,3,90,0,0,126
3,3,10,90,1,0,126
4,4,8,90,1,0,125
5,5,12,90,1,1,125
6,6,7,90,1,0,126
7,7,13,90,1,1,126
8,8,13,90,1,1,127
9,9,24,90,3,1,128


In [162]:
names(get_gameweek_history(20))

31-element Vector{String}:
 "assists"
 "bonus"
 "bps"
 "clean_sheets"
 "creativity"
 "element"
 "fixture"
 "goals_conceded"
 "goals_scored"
 "ict_index"
 "influence"
 "kickoff_time"
 "minutes"
 ⋮
 "saves"
 "selected"
 "team_a_score"
 "team_h_score"
 "threat"
 "total_points"
 "transfers_balance"
 "transfers_in"
 "transfers_out"
 "value"
 "was_home"
 "yellow_cards"